# AES

# Chiffrement d'un message avec AES et une clé de 256 bits.

In [55]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
from os import urandom

def encrypt(key, message):
    """Chiffre un message avec AES et une clé de 256 bits.

    Args:
        key (bytes): Clé de 256 bits.
        message (bytes): Message à chiffrer.

    Returns:
        bytes: Message chiffré.
    """
    iv = urandom(16)
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(message) + padder.finalize()
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()
    return iv + ciphertext

def decrypt(key, ciphertext):
    """Déchiffre un message avec AES et une clé de 256 bits.

    Args:
        key (bytes): Clé de 256 bits.
        ciphertext (bytes): Message chiffré.

    Raises:
        ValueError: Le message n'est pas de la bonne taille, il doit être un multiple de 16 octets.

    Returns:
        str: Message déchiffré.
    """
    if len(ciphertext[16:]) % 16 != 0:
        raise ValueError(f'Le message ({len(ciphertext[16:])} octets) n\'est pas de la bonne taille, il doit être un multiple de 16 octets')
    cipher = Cipher(algorithms.AES(key), modes.CBC(ciphertext[:16]), backend=default_backend())
    decryptor = cipher.decryptor()
    decrypted = decryptor.update(ciphertext[16:]) + decryptor.finalize()
    unpadder = padding.PKCS7(128).unpadder()
    unpadded = unpadder.update(decrypted) + unpadder.finalize()
    return unpadded.decode()

key = "01" * 128
key = int(key, 2).to_bytes(len(key) // 8, byteorder='big')

message = b'Test de chiffrement AES avec CBC et un padding PKCS7'
ciphertext = encrypt(key, message)
print("Le message chiffré :", ciphertext)

decrypted = decrypt(key, ciphertext)
print("Le message déchiffré :", decrypted)

Le message chiffré : b'\xddN\x19=\x13I0:\xe1\xfe\xf1\x80B\x7f\xf8\x89r\x0e\xb4\xa2,n\xd5\x17\x96\xd4ti[\xb8\xdf\xfci\xdb\x01\x07\xc4\x19\xa5M9\xed\x16\x1c"\xbdO\x03<\x0beg\n\xa9\xa2\x8c\xdfR\x83\x89w\x92\xb9\x92\x0eG\xe7c]\xc4\xbb\x1b\xa9\x93\x9a<1\x92-\xe8'
Le message déchiffré : Test de chiffrement AES avec CBC et un padding PKCS7


# Recherche de la clé dans les images

In [56]:
from PIL import Image

def dechiffrer_image(path):
    image = Image.open(path)
    pixels = image.load()
    (largeur, hauteur) = image.size
    return ''.join([str(pixels[x, y] % 2) for y in range(hauteur) for x in range(largeur)])[:64]

key = dechiffrer_image('data/rossignol2.bmp')
print(f'La clé est en binaire : {key} (en hexadécimal : {hex(int(key, 2))})')

La clé est en binaire : 1110011101101101001100010011111110010010101110011001000001001100 (en hexadécimal : 0xe76d313f92b9904c)
